In [117]:
def avg_per_course():
    print("Here are the average scores in each course: ")
    avg_scores = main_df.groupby('course')['score'].mean().reset_index()
    print(avg_scores.to_string(index=False))

In [124]:
#The user enter the student's name and gets his/her avg grade
def avg_student():
    while True:
        student_name=input(f'Student name: (student for example:{student[random.randint(0,99)]})'.title().strip())
        if student_name in student.values:
            print(f"{student_name} has average score of {main_df.loc[main_df['student']==student_name]['score'].mean()}")
        else:
            print(f"There isn't student called {student_name}")
        next_action= input("Do you want to check another student's average score? (yes/no): ").strip().lower()
        # TODO: support lower case- ido already being DONE by using .lower
        if next_action != 'yes':
            break

In [123]:
def grade_groups(course_name,scores_groups_bins,scores_groups_labels):    
    # TODO: you can use main_df.query("Subect == @course_name") - it's more readable, ido:unneccesery
    # TODO: don't call stuff df - it's not descriptive DONE
    scores = main_df.query(f"course=='{course_name}'")['score']
    # TODO: don't call stuff resutls - it's not descriptive DONE
    scores_by_groups = scores.groupby(pd.cut(scores, bins=scores_groups_bins, labels=scores_groups_labels)).size()
    print('Scores distribution by groups:')
    
    # TODO: if it's a series you can do series.iteritems() instead of converting to a dataframe DONE
    for index, row in scores_by_groups.items():
        print(f"{index}: {row}")

def general_info(course_name):
    # TODO: why not show all the courses together? main_df.groupby('course').describe() ido-Correct, DONE
    # TODO use pd.set_option("display.precision", 2) at the start of the script with the imports - it's more readable DONE
    info_course=main_df.query(f"course=='{course_name}'").describe()
    info_course.index=['Number of tests:','Average:','Std:','Lowest grade:','25% grade','50% grade','75% grade','Highest grade']
    info_general=main_df.describe()
    info_general.index=['Number of tests:','Average:','Std:','Lowest grade:','25% grade','50% grade','75% grade','Highest grade']
    print(f"\nCourse's statistics \n {info_course}")
    print(f"\nAll courses's statistics \n {info_general}")

# TODO: use seaborn sns.set_style('whitegrid) at the start of the script with the imports - it's more readable DONE
def grade_historgram(course_name):
    main_df[main_df['course']==course_name]['score'].plot(kind='hist',title=f"{course_name}'s' scores distribution")
    plt.show()

#provide per course: grade's histogram,grade distribution by groups and statistics using describe's function
def statistics_by_course():
    global global_vars
    print("For which course you want to get info on:\n{}".format('\n'.join(course.values)))
    course_name=input(f'course: ').title().strip()
    if course_name not in global_vars['course'].values:
        print(f"There isn't course called {course_name}, return to main menu and add {course_name} as course and add the students' scores")
    else:
        global_vars['course_name']=course_name
        grade_historgram(course_name)
        grade_groups(course_name=course_name,scores_groups_bins=global_vars['scores_groups_bins'],scores_groups_labels=global_vars['scores_groups_labels'])
        general_info(course_name)

In [135]:
#display top 5 students by z-score's calculation (each course gets equal weight) and their class ranks in each course

def top_5_students():
    global main_df
    #Create copy of main_df that will contain z_score and class rank
    top_students_df=main_df.copy()

    # TODO: use a pandas function for this like df.rank DONE
    #add class rank column to existing df
    top_students_df=rank_in_class(top_students_df)

    # TODO: use sklearn.preprocessing.StandardScaler() to calculate z-score DONE
    #add z_score column to existing df
    top_students_df=calculate_z_score(top_students_df)

    print(f"""Here are the top 5 students based on Z-score calculation and their class' rank in each course
    (For example: position 8 in Art means that this student got the 8th highest grade in Art)\n {top_students_df}""")        
    

#student's class rank in each course
def rank_in_class(df):
    # TODO: df.rank instead DONE
    df['class_rank']=df.groupby('course')['score'].rank(ascending=False, method='min').astype(int)
    return df


#student's z-score in each course
def calculate_z_score(df):
    
    scaler = StandardScaler()
    
    def calculate_course_z_score(course_df):
        course_z_scores=scaler.fit_transform(course_df[['score']])
        return pd.Series(course_z_scores[:,0],index=course_df.index,name='z_score')
    
    df['z_score']=df.groupby('course').apply(calculate_course_z_score).reset_index(drop=True)
    
    def calculate_total_z_score(df):
        total_z_score=df.groupby('student')['z_score'].sum().reset_index().rename(columns={'z_score':'total_z_score'})
        df = df.merge(total_z_score, on='student', how='left')
        return df
                 
    df=calculate_total_z_score(df)    
    df=df.drop('z_score',axis=1)
    #use pivot to get each course's class rank in seperate column
    df=df.pivot(
        columns='course',
        index=['student','total_z_score'],
        values='class_rank').sort_values(by='total_z_score',ascending=False).reset_index()
    return df.head().to_string(index=False)
    
    # TODO: lines like this should be multilines DONE
    # df_z_score=df_z_score.pivot(
    #     columns='course',
    #     index=['Student','Total Z-score'],
    #     values='class_rank')\
    #         .sort_values(by='Total Z-score',ascending=False).reset_index()